# INNIO Fleet Analysis

In [173]:
import pandas as pd
pd.options.mode.chained_assignment = None # default warn => SettingWithCopyWarning
import numpy as np
import matplotlib.pyplot as plt
from pprint import (pprint as pp, pformat as pf)
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)
from IPython.display import HTML, display
import ipywidgets as widgets

import dmyplant2
dmyplant2.cred()
mp = dmyplant2.MyPlant(3600)

### Filter Engines from installed fleet 

In [179]:
def sfun(x):
    return all([
            ("Forsa Hartmoor" in str(x['IB Site Name'])), 
            #("BMW" in str(x['IB Site Name'])), 
            (x['OperationalCondition'] != 'Decommissioned')
        ])

In [182]:
res = mp.search_installed_fleet(sfun).drop('index', axis=1)
res = res.sort_values(by = "Engine ID",ascending=True).reset_index(drop='index')
res.T;

In [183]:
ddl = pd.DataFrame(res['serialNumber'] + ' - ' + res['IB Site Name'] + ' ' + res['Engine ID'])[0].to_list()
ddl = [(m, i) for i, m in enumerate(ddl)]
dl=widgets.Dropdown(options = ddl,value=0,description='Engine:',layout={'width':'max-content'},disabled=False)
display(dl)

Dropdown(description='Engine:', layout=Layout(width='max-content'), options=(('1486144 - Forsa Hartmoor M01', …

In [177]:
rmodes = ['???','OFF','MANUAL','AUTO']; mw = [] ; modes = []
for mm in rmodes:
    mw.append(widgets.Checkbox(value=False, description='Mode: ' + mm))
rsucc = [True,False]; sw = []; success=[]
for rs in rsucc:
    sw.append(widgets.Checkbox(value=False, description='Success: ' + str(rs)))
display(widgets.HBox(mw+sw))



In [65]:
e=dmyplant2.Engine.from_fleet(mp,motor:=res.iloc[dl.value])
pd.DataFrame.from_dict(e.dash, orient='index').T

,Name,Engine ID,Design Number,Engine Type,Engine Version,P,P_nom,BMEP,serialNumber,id,Count_OpHour,val start,oph@start,oph parts,LOC
0,Forsa Hartmoor M05,M05,AL89,624,H12,24,4582.1,24.5,1486197,159400,180.0,2022-01-22,110,70,0.1325


In [66]:
from dfsm import msgFSM
#fsm = msgFSM(e, skip_day=7) #funktioniert nicht
fsm = msgFSM(e, p_from=motor['Commissioning Date'])
fsm.run() # run Finite State Machine

fsm_data = []
fsm_data.append({'engine':e, 'fsm':fsm})

Using '9047 target load reached' Message.


FSM: 100%|████████████████████████| 3279/3279 [00:00<00:00, 11670.49 messages/s]


In [82]:
res = fsm_data[0]
res['result'] = pd.DataFrame(fsm_data[0]['fsm']._starts)
fsm.summary(res)
rdf = res['result']

,From,To,Days,Target Load detection
Interval,22.01.2022,24.02.2022,33,Message '9047 target load reached' .


,Starts,successful,%
???,10,10,100
OFF,0,0,0
MANUAL,9,1,11
AUTO,28,23,82
ALL,47,34,72


In [170]:
modes = [rmodes[i] for i,v in enumerate(mw) if v.value]
modes = rmodes if not modes else modes
success = [rsucc[i] for i,v in enumerate(sw) if v.value]
success = rsucc if not success else success

rda = rdf[:].reset_index(drop='index')
rda['count_alarms'] = rda.apply(lambda x: len(x['alarms']), axis=1)
rda['count_warnings'] = rda.apply(lambda x: len(x['warnings']), axis=1)
rda = rda[((rdf['mode'].isin(modes)) & (rdf['success'].isin(success)))].reset_index(drop='index')
print(modes, success)

['MANUAL', 'AUTO'] [True]


In [171]:
# special filters can be added like ... 
#rda = rdf[((rdf['synchronize'] < 300.0) & (rdf['startpreparation'] < 300.0))].reset_index(drop='index')

#rda = rda[((rda['count_warnings'] > 0) | (rda['count_alarms'] > 0))].reset_index(drop='index')
# display the filterd & selected data
rda[fsm.filter_content + ['count_alarms', 'count_warnings']].round(2)

,success,mode,startpreparation,starter,hochlauf,idle,synchronize,loadramp,cumstarttime,targetoperation,count_alarms,count_warnings
0,True,AUTO,77.79,3.94,24.15,4.65,45.01,120.38,275.91,0 days 03:43:28,0,0
1,True,AUTO,78.01,4.13,22.00,4.45,34.46,133.64,276.69,0 days 01:33:27,0,0
2,True,AUTO,79.47,4.44,24.46,4.75,26.90,135.15,275.17,0 days 00:23:28,0,0
3,True,AUTO,77.90,4.14,20.38,4.44,47.98,120.81,275.66,0 days 01:03:30,0,0
4,True,AUTO,77.80,3.83,19.58,4.43,38.36,132.27,276.27,0 days 00:43:28,0,0
5,True,AUTO,77.68,3.53,20.38,4.44,34.75,135.59,276.38,0 days 00:53:30,0,0
6,True,AUTO,60.65,3.63,19.36,4.24,36.72,151.57,276.16,0 days 00:53:33,0,0
7,True,AUTO,60.72,4.04,18.79,4.25,25.27,163.26,276.33,0 days 00:23:27,0,0
8,True,AUTO,75.37,4.14,19.57,4.44,4.74,166.95,275.21,0 days 02:43:31,0,0
9,True,AUTO,78.39,3.74,24.21,4.94,2.32,161.61,275.20,0 days 01:23:25,0,0


In [ ]:
startversuch = rda.iloc[3]; fsm._pre_period = 15
von_dt=pd.to_datetime(startversuch['starttime']); von=int(von_dt.timestamp() - fsm._pre_period)
bis_dt=pd.to_datetime(startversuch['endtime']); bis=int(bis_dt.timestamp())
cycle = 1 #sec.
data = fsm.get_period_data(von, bis, cycletime=cycle)
ftitle = f"{fsm._e} ----- Start {startversuch.name} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')}"
print(f"von:{von_dt} = {von} bis:{bis_dt} = {bis}")

In [ ]:
dset = [
    {'col':['Power_PowerAct'], 'ylim':(0,5000)},
    {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500]}
]
dmyplant2.dbokeh_chart(data, dset, title=ftitle, figsize=(16,8))

In [ ]:
lines = startversuch[fsm.vertical_lines_times]
[lines[:i+1].sum() for i,v in enumerate(lines)]

In [ ]:
import arrow
def xplot_cycle(rec, max_length=None, cycletime=None, *args, **kwargs):
    t0 = int(arrow.get(rec['starttime']).timestamp() * 1e3 - fsm._pre_period * 1e3)
    t1 = int(arrow.get(rec['endtime']).timestamp() * 1e3)
    if max_length:
        if (t1 - t0) > max_length * 1e3:
            t1 = int(t0 + max_length * 1e3)
    data = fsm.load_data(cycletime, tts_from=t0, tts_to=t1)
    (ax, ax2, idf) = fsm._plot(
        data[
            (data['time'] >= t0) & 
            (data['time'] <= t1)],        
            *args, **kwargs
        )
    return ax,ax2,idf

def org_add_lines(rec, ax):
    duration = 0.0
    for k in rec[fsm.vertical_lines_times].index:
        dtt=rec[k]
        if dtt == dtt:
            ax.axvline(arrow.get(rec['starttime']).shift(seconds=duration).datetime, color="red", linestyle="dotted", label=f"{duration:4.1f}")
            duration = duration + dtt
        else:
            break
    ax.axvline(arrow.get(rec['starttime']).shift(seconds=duration).datetime, color="red", linestyle="dotted", label=f"{duration:4.1f}")

lines = startversuch[fsm.vertical_lines_times]
def add_lines(rec, ax):
    duration = 0.0
    for k in rec[fsm.vertical_lines_times].index:
        dtt=rec[k]
        if dtt == dtt:
            ax.axvline(arrow.get(rec['starttime']).shift(seconds=duration).datetime, color="red", linestyle="dotted", label=f"{duration:4.1f}")
            duration = duration + dtt
        else:
            break
    ax.axvline(arrow.get(rec['starttime']).shift(seconds=duration).datetime, color="red", linestyle="dotted", label=f"{duration:4.1f}")


def add_table(summary, ax, *args, **kwargs):
    """
    available options for loc:
    best, upper right, upper left, lower left, lower right, center left, center right
    lower center, upper center, center, top right,top left, bottom left, bottom right
    right, left, top, bottom
    """
    ax.table(
        cellText=summary.values, 
        colWidths=[0.1]*len(summary.columns),
        colLabels=summary.columns,
        cellLoc='center', 
        rowLoc='center',
        *args, **kwargs)
        #loc='upper left')
    return idf

ax, ax2, idf = xplot_cycle(startversuch, max_length=None, ylim=(0,2500), cycletime=1, style='.-', figsize=(22,12), title=ftitle);
add_lines(startversuch, ax)

summary = pd.DataFrame(startversuch[fsm.filter_times], dtype=np.float64).round(2).T
add_table(summary, ax, loc='upper left')

plt.show()

In [ ]:
for i,startversuch in rda.iterrows():
    r_summary = pd.DataFrame(startversuch[fsm.filter_times], dtype=np.float64).round(2).T
    fsm.plot_cycle(startversuch, max_length=None, ylim=(0,2500), cycletime=1, style='.-', figsize=(18,12), 
            title=f"{fsm._e} ----- Start {startversuch.name} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')}");
    plt.show();

In [ ]:
rda.describe().round(2).T;

In [ ]:
longer_than_300s = rda[fsm.filter_period + fsm.filter_content][rdf['cumstarttime'] > 300.0]
longer_than_300s[fsm.filter_content].round(2);

In [ ]:
#for i, r in longer_than_300s.iterrows():
#    fsm.plot_cycle(r, max_length=20*60, ylim=(0,5000), title=f"{fsm._e}{r['starttime'].round('S')}: cumstarttime: {r['cumstarttime']}s");

In [ ]:
longer_than_300s.describe().round(2).T;

In [ ]:
load_ramp_less_than_100s = rda[fsm.filter_period + fsm.filter_content][rdf['load-ramp'] < 100.0]
lfsm = load_ramp_less_than_100s[fsm.filter_content].reset_index(drop='index').round(2)
lfsm

In [ ]:
rda.reset_index(drop='index').iloc[10];

In [ ]:
#r = load_ramp_less_than_100s.reset_index(drop='index').iloc[0]
for i in np.arange(0,5):
    r = rda.reset_index(drop='index').iloc[i]
    r_summary = pd.DataFrame(r[fsm.filter_times], dtype=np.float64).round(2).T
    fsm.plot_cycle(r, max_length=20*60, ylim=(0,2500), cycletime=1, marker=None, figsize=(16,10), title=f"{fsm._e} {r['mode']} {r['starttime'].round('S')}");
    plt.show()

In [ ]:
for i, r in load_ramp_less_than_100s.reset_index(drop='index').iterrows():
    fsm.plot_cycle(r, max_length=8*60, ylim=(0,2500), cycletime=1, marker=None,figsize=(20,12), title=f"{i:3d} - {fsm._e} {r['starttime'].round('S')}")
    plt.show()


In [ ]:
load_ramp_less_than_100s.describe().round(2).T

In [ ]:
rda[['start-preparation','starter','hochlauf','idle','synchronize','cumstarttime','load-ramp','target-operation']].hist(figsize = (20,12), bins=50, layout=(3,3));
#rda.hist(figsize = (20,12), bins=100, layout=(4,2));

In [ ]:
#rda.plot.box(subplots=True, grid = True, figsize=(24,10), layout=(1,7));
#rda.plot.box(subplots=True, grid = True, figsize=(24,10), layout=(1,7), sym='');

In [ ]:
nalarms = []
ct = 0
ct2 = 0
mini = 0
maxi = 1
for i,c in rdf.iterrows():
    if len(c['alarms']) > 0 and not c['success']:
        ct += 1
        print(f"\nStartversuch: {i}, Success: {c['success']}")
        for a in c['alarms']:
            nalarms.append(a['msg'])
            _txt = f"{ct2} {c['mode']:15} {a['state']:20} {a['msg']['timestamp']} {pd.to_datetime(int(a['msg']['timestamp'])*1e6).strftime('%d.%m.%Y %H:%M:%S')} {a['msg']['name']} {a['msg']['message']}"
            print(_txt)
            if ct2 >= mini and ct2 <= maxi:
                fsm.plot_cycle(c, ylim=(0,2500), cycletime=1, marker=None,figsize=(20,12), title=f"{i:3d} - {fsm._e} {_txt}")
                plt.show()
            ct2 += 1

print(f"""
***********************************
** {ct:3} nicht erfolgreiche Starts **
***********************************
""")

In [ ]:
al = pd.DataFrame(fsm._pareto(nalarms))
fig = plt.figure();
color = 'purple'

if not al.empty:
    al['msg'] = al['msg'] + ' (' + al['name'] + ')'
    al.set_index('msg').sort_values(by = "anz",ascending=True).plot.barh(y=['anz'],figsize=(16,len(al) / 3.8), color=[color], position = 1.0, grid=True, title='Alarms in not successful Starts');
    plt.plot();

In [ ]:
phases = [['start-preparation','starter','hochlauf','idle','synchronize','load-ramp','target-operation'],'start-preparation','starter','hochlauf','idle','synchronize','load-ramp','target-operation']
fig = plt.figure();
color = 'red'

for phase in phases:
    al = fsm.alarms_pareto(phase)[:30]
    if not al.empty:
        al['msg'] = al['msg'] + ' (' + al['name'] + ')'
        al.set_index('msg').sort_values(by = "anz",ascending=True).plot.barh(y=['anz'],figsize=(16,len(al) / 3.8), color=[color], position = 1.0, grid=True, title=' | '.join(phase) if type(phase) == list else phase);
        plt.plot();


In [ ]:
phases = [['start-preparation','starter','hochlauf','idle','synchronize','load-ramp','target-operation'],'start-preparation','starter','hochlauf','idle','synchronize','load-ramp','target-operation']
fig = plt.figure();
color = 'orange'

for phase in phases:
    al = fsm.warnings_pareto(phase)[:30]
    if not al.empty:
        al['msg'] = al['msg'] + ' (' + al['name'] + ')'
        al.set_index('msg').sort_values(by = "anz",ascending=True).plot.barh(y=['anz'],figsize=(16,len(al) / 3.8), color=[color], position = 1.0, grid=True, title=' | '.join(phase) if type(phase) == list else phase);
        plt.plot();


In [ ]:
mfn = e._fname + '_messages.txt'
fsm.save_messages(mfn)
print(mfn)

In [ ]:
any(fsm._messages['name'] == '9047')

In [ ]:
#fsm.store()